Google mount + setting


- glob
  - 인자로 받은 패턴과 이름이 일치하는 모든 파일과 directory의 list return
  - \* : 모든 파일과 디렉터리 i.e. glob('\*.exe')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
from glob import glob

drive_project_root = 'drive/MyDrive/data/'
sys.path.append(drive_project_root)

image_root = 'drive/MyDrive/data/images/'
anno_root = 'drive/MyDrive/data/annotations/'

image_dir = image_root
bbox_dir = anno_root + 'xmls/'    # bounding box
seg_dir = anno_root + 'trimaps/'  # segmentation map

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

!ls

In [ ]:
pip install opencv-python

In [ ]:
import math
import random 

import cv2                                 # image를 읽기 위한 open cv library
import xml.etree.ElementTree as et         # xml 파일을 parsing 하기 위한 library
from matplotlib.patches import Rectangle   # Bounding box를 그리기 위함

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style('whitegrid')

# 1. Dataset EDA
- data : Google [oxford pet dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/)
- Download 'dataset', 'groundtruth data'

In [ ]:
df = pd.read_csv(anno_root+'/list.txt', skiprows=6, delimiter=" ", header=None)
df.columns = ['file_name', 'id', 'species', 'breed']
df

### 1.1 숫자 분포 확인

In [ ]:
def num_plot(value_counts):
    plt.bar(range(len(value_counts)), value_counts.values, align='center')
    plt.xticks(range(len(value_counts)), value_counts.index.values)
    plt.tight_layout()    # padding(여백) 조정
    plt.show()

In [ ]:
value_counts = df['species'].value_counts().sort_index()  # 1: 고양이, 2: 강아지
print(value_counts)
print('\nrange: ',range(len(value_counts)))
print('\nvalues: ',value_counts.values)

num_plot(value_counts)

- 각 class id에 대해 대략 200장 씩 이미지가 있다는 것 확인

In [ ]:
value_counts = df['id'].value_counts().sort_index()
num_plot(value_counts)

- 고양이와 개 각각에 대한 class id 구성
- 고양이는 12가지, 강아지는 25가지 종류가 있음을 확인

In [ ]:
value_counts = df[df['species'] == 1]['breed'].value_counts().sort_index()  # 고양이만
num_plot(value_counts)

In [ ]:
value_counts = df[df['species'] == 2]['breed'].value_counts().sort_index()  # 강아지만
num_plot(value_counts)

### 1.2 파일 읽기

In [ ]:
image_files = glob(image_dir + '*.jpg')  # 경로 읽기
print(len(image_files))

image_files[:10]   # 경로가 제대로 읽혔는지 확인

In [ ]:
seg_files = glob(seg_dir + '*.png')
print(len(seg_files))

seg_files[:10]

In [ ]:
bbox_files = glob(bbox_dir + '*.xml')
print(len(bbox_files))     # image_files, seg_files보다 적음 -> 모든 이미지에 대해 head ROI가 제공되는 것은 아니라는 것

bbox_files[:10]

### 1.3 Head ROI 시각화
- ROI :  Region of Interest

In [ ]:
# 임의의 이미지 파일 경로 하나 가져와서 이에 해당하는 bounding box의 경로 찾기
image_path = image_files[80]
bbox_path = image_path.replace(image_dir, bbox_dir).replace('jpg', 'xml')

# image 파일을 open cv로 읽기
# 주의점 : RGB가 아니라 BGR 형식으로 읽힘
image = cv2.imread(image_path)                 # ★ path에 한국어 경로 있으면 안 읽힐 수 있음
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # BGR 형식을 RGB로 바꿔줄 것

# xml파일 parsing 하기
tree = et.parse(bbox_path)

# bounding box 좌표는 object (bounding box) tag 아래 있었음
xmin = float(tree.find('./object/bndbox/xmin').text)
xmax = float(tree.find('./object/bndbox/xmax').text)
ymin = float(tree.find('./object/bndbox/ymin').text)
ymax = float(tree.find('./object/bndbox/ymax').text)

# 가로, 세로 길이
rect_x = xmin
rect_y = ymin
rect_w = xmax - xmin
rect_h = ymax - ymin

# Head ROI (bounding box 객체 만들기)
# cv2.Rectangle(image, pt1 시작점 좌표, pt2 종료점 좌표, color, thickness, lineType, shift)
rect = Rectangle((rect_x, rect_y), rect_w, rect_h, fill=False, color='red')
plt.axes().add_patch(rect)
plt.imshow(image)

plt.show()

### 1.4 SEG map
- 보통 3가지 채널이 아닌 하나의 채널에 정보가 모두 저장되어 있음 (grayscale)

In [ ]:
image_path = image_files[90]
seg_path = image_path.replace(image_dir, seg_dir).replace('jpg', 'png')

image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
seg_map = cv2.imread(seg_path, cv2.IMREAD_GRAYSCALE)  # Grayscale 이미지를 읽어온다는 것 알려줌

# side by side 출력
plt.figure(figsize=(10, 10))
plt.subplot(1,2,1)
plt.imshow(image)
plt.subplot(1,2,2)
plt.imshow(seg_map)

plt.show()

---
# 2. 이미지 분류 (k-fold)
- 'fold' 컬럼 생성 : validation set으로 index → 향후 작업하기 편함
- 'fold' 별로 데이터가 균등하게 뽑혔는지 항상 주의
  - training에 요크셔테리어가 없으면 test set에서 요크셔테리어 분류 못함
  - ★ 방법 : sklearn의 StratifiedKFold 사용

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

### 2.1 KFold 사용하기

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# fold column 생성 → -1로 초기화
df['fold'] = -1

for idx, (train, val) in enumerate(kf.split(df), 1):
    print(idx, train, val, len(val))
    df.loc[val, 'fold'] = idx     # 'fold' 컬럼, 'val' 행에 index 저장 (1~5)

# Check dataset
print(df[:5])
print(len(df[df['fold']==1]))  # 첫 번째 fold에 대한 validation set
print(len(df[df['fold']!=1]))  # 첫 번째 fold에 대한 train set

- 강아지와 고양이 품종이 균등하게 분포되어 있는지 시각화로 확인

In [ ]:
value_counts = df[df['fold'] != 5]['id'].value_counts().sort_index()   # 5번째 fold의 training data

plt.bar(range(len(value_counts)), value_counts.values, align='center')
plt.xticks(range(len(value_counts)), value_counts.index.values)
plt.tight_layout()
plt.show()

### 2.2 StratifiedKFold 사용하기

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

df['fold'] = -1

for idx, (train, val) in enumerate(skf.split(df, df['id']), 1):  # 'id' (품종)을 최대한 균등하게 하겠다
    print(train, val, len(val))
    df.loc[val, 'fold'] = idx

- 강아지와 고양이 품종이 균등하게 분포되어 있는지 시각화로 확인

In [ ]:
value_counts = df[df['fold'] != 5]['id'].value_counts().sort_index()   # 5번째 fold의 training data

plt.bar(range(len(value_counts)), value_counts.values, align='center')
plt.xticks(range(len(value_counts)), value_counts.index.values)
plt.tight_layout()
plt.show()

### 2.3 결과 저장

In [ ]:
df.to_csv(drive_project_root+'kfolds.csv', index=False)

---
# 3. Data Loader
- keras로 data loader를 작성할 때는 sequence를 사용함
  - 일반 generator를 사용하는 것보다 multi-processing에 더 적합하기 때문 (keras 공식문서에서도 sequence 사용을 권장)
- sequence 함수
  - 반드시 있어야 하는 것 : len, getitem
    - len : 한 epoch에 몇 개의 batch가 있는지
    - getitem : 하나의 batch 생성
  - optional : on_epoch_end

### 3.1 Oxford pet data에 대한 sequence 함수 작성하기

In [ ]:
class DataGenerator(keras.utils.Sequence):
    # fold : 5가지 중 어떤 fold를 사용할지
    def __init__(self, batch_size, csv_path, fold, image_size, mode='train', shuffle=True):
        self.batch_size = batch_size
        self.df = pd.read_csv(csv_path)
        self.fold = fold
        self.image_size = image_size
        self.mode = mode
        self.shuffle = shuffle        
        
        # 필요한 데이터만 남기기
        if self.mode == 'train':
            self.df = self.df[self.df['fold'] != self.fold]  # training
        elif self.mode == 'val':
            self.df = self.df[self.df['fold'] == self.fold]  # validation
            
        # Remove invalid files
        invalid_filenames = [
            'Egyptian_Mau_14',
            'Egyptian_Mau_139',
            'Egyptian_Mau_145',
            'Egyptian_Mau_156',
            'Egyptian_Mau_167',
            'Egyptian_Mau_177',
            'Egyptian_Mau_186',
            'Egyptian_Mau_191',
            'Abyssinian_5',
            'Abyssinian_34',
            'chihuahua_121',
            'beagle_116'            
        ]
        self.df = self.df[-self.df['file_name']. \
                         isin(invalid_filenames)]
        
        # 학습을 시작하는 첫 epoch에서도 shuffle을 하고 싶은 경우
        self.on_epoch_end()  # 원치 않는 경우 이 부분 코드는 빼기
        
    def __len__(self):
        # 나눠 떨어지지 않는 경우 남는 데이터가 생김 -> 올림 필요
        return math.ceil(len(self.df) / self.batch_size)
    
    # idx : 몇 번째 batch를 return 하고 싶은지 ex) 첫 번째 : 0
    def __getitem__(self, idx):
        strt = idx * self.batch_size
        fin = (idx + 1) * self.batch_size
        data = self.df.iloc[strt:fin]
        
        batch_x, batch_y = self.get_data(data)   # image, label
        return np.array(batch_x), np.array(batch_y)
    
    # data 내 모든 image와 label return
    def get_data(self, data):
        
        batch_x = []
        batch_y = []
        
        # r : csv 파일의 값들
        for _, r in data.iterrows():  # index, row 전체
            file_name = r['file_name']
            image = cv2.imread(f'{image_root}{file_name}.jpg')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # input image들의 size가 각각 다 다르기 때문에 같은 size로 resize 필요
            # if not, 하나의 batch로 묶어서 array 만들기 불가
            image = cv2.resize(image, (self.image_size, self.image_size))
            image = image / 255.  # rescale : 0~255 px -> 0~1
            
            # 개와 고양이 이진분류
            label = int(r['species']) - 1  # 현재 데이터 : 고양이 1, 개 2 -> -1 -> 고양이 0, 개 1
            
            batch_x.append(image)
            batch_y.append(label)
            
        return batch_x, batch_y
            
    # callback 함수 : 매 epoch이 끝날 때마다 도는 함수
    # ex) shuffle 하는 함수를 넣으면 한 epoch이 끝날 때마다 데이터 자동 shuffle
    def on_epoch_end(self):
        if self.shuffle:
            # frac : 몇 퍼센트의 데이터를 shuffle 할지 (1 -> 100% : 전체 데이터)
            # drop=True : 기존 인덱스를 버리고 재배열
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [ ]:
csv_path = drive_project_root+'kfolds.csv'
train_generator = DataGenerator(
    batch_size=9,
    csv_path=csv_path,
    fold=1,
    # image_size=256,  # 256×256
    image_size=16,     # 자꾸 메모리 오류 발생해서 크기 변경
    mode='train',
    shuffle=True
)

### 3.2 데이터 개수 확인

- 전체 데이터 : 7,390개
- 5-fold + train mode -> 전체 데이터의 80% 정도가 있어야 함 (5.8천 여 개)

In [ ]:
print(len(train_generator))
print(len(train_generator) * 9)   # 9 : batch size

### 3.3 data generator가 이미지를 제대로 load 하는 지 확인

In [ ]:
class_name = ['Cat', 'Dog']  # 고양이 0, 강아지 1

for batch in train_generator:
    X, y = batch   # X: image, y: label
    plt.figure(figsize=(10, 10))
    
    for i in range(9):                # 9개 이미지 살펴보기
        ax = plt.subplot(3, 3, i+1)   # 총 3*3개 plot, i+1 번째에 그리기
        plt.imshow(X[i])
        plt.title(class_name[y[i]])
        plt.axis('off')   # on : 사진 사이에 가로세로 grid 생김
    
    # 첫 번째 batch만 확인할 예정
    break

- label

In [ ]:
for batch in train_generator:
    # X: image, y: label
    X, y = batch
    print(y)
    break

---
# 4. Model 구현

### 4.1 Sequential 모델 구현
- simple한 모델들만 구현 가능
- sequential class 단독으로 사용하기 보단, model의 block 단위를 간단하게 구현할 때 사용

MaxPooling vs Global MaxPooling
- Max-pooling layer gave the largest value in a certain subarea as an output
  - 각각의 filter에 대해 주어진 kernel 안에서 최대값 return -> kernel 사이즈가 작아짐
- While the global max-pooling did this in the whole area


In [ ]:
def get_sequential_model(input_shape):
    
    # layer를 순서대로 쌓기
    model = keras.Sequential(
        [
            layers.Input(input_shape),  # input

            # CNN에서 많이 쓰이는 heuristic
            # MaxPooling 거쳐서 이미지 사이즈가 반으로 감소
            # 그만큼 다음 layer에서 filter 사이즈를 키움 (64 -> 128)
            # VGG 때부터 많이 쓰인 방식
            
            # 1st Conv block
            # convolution을 적용하면 image size가 2px 씩 작아짐
            # 여러 번 거치면 완전 작아짐
            # 이를 방지하기 위해 padding 사용 (same : input과 output의 크기가 일치)
            layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),  # filter size 64, kerner size 3*3
            layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.5),
                    
            # 2nd Conv block
            layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
            layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            
            # Classifier
            layers.GlobalMaxPool2D(),
            layers.Dense(128, activation='relu'),
            layers.Dense(1, activation='sigmoid')  # 이진분류 (개, 고양이)            
        ]
    )
    
    return model

input_shape = (256, 256, 3)
model = get_sequential_model(input_shape)

model.summary()

### 4.2 Functional API 모델 구현
- keras 공식 모델이 저장되어 있는 keras applications에 있는 모델들
- Sequential과 달리 multi input/output, layer sharing 등 특별한 제약 없이 다양한 기능 사용 가능

In [ ]:
def get_functional_model(input_shape):
    
    # Sequential과 달리 사용자가 직접 모든 layer의 결과를 저장하고 다음 레이어에 넘겨줘야 함
    inputs = keras.Input(input_shape)
    
    # 1st Conv block
    x = layers.Conv2D(64, 3, strides=1, activation='relu', padding='same')(inputs)
    x = layers.Conv2D(64, 3, strides=1, activation='relu', padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)

    # 2nd Conv block
    x = layers.Conv2D(128, 3, strides=1, activation='relu', padding='same')(x)
    x = layers.Conv2D(128, 3, strides=1, activation='relu', padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    # Classifier
    x = layers.GlobalMaxPool2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)  # 이진분류 (개, 고양이)
    
    model = keras.Model(inputs, outputs)
    
    return model

input_shape = (256, 256, 3)
model = get_functional_model(input_shape)

model.summary()

### 4.3 Model Subclassing
- pytorch 경험자들은 이 방법이 가장 수월할 듯
- keras의 model class를 상속받아서 필요한 method를 override
  - functional API와 달리, model.fit, model.evaluate, model.predict를 override 해서 customize 가능
  - override 할 필요가 없는 경우 굳이 model subclassing보다는 functional API 사용하는 것으로 충분
- model.summary() 결과가 다른 모델 대비 간단한 이유
  - keras에서는 하나의 block을 하나의 layer로 생각함
  - = 모델에는 3가지 layer만 있는 것처럼 인지됨

In [ ]:
class SimpleCNN(keras.Model):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        self.conv_block_1 = keras.Sequential(
            [
                layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
                layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
                layers.MaxPool2D(),
                layers.BatchNormalization(),
                layers.Dropout(0.5)          
            ], name='conv_block_1'
        )
        
        self.conv_block_2 = keras.Sequential(
            [
                layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
                layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
                layers.MaxPool2D(),
                layers.BatchNormalization(),
                layers.Dropout(0.3)
            ], name='conv_block_2'
        )
        
        self.classifier = keras.Sequential(
            [
                layers.GlobalMaxPool2D(),
                layers.Dense(128, activation='relu'),
                layers.Dense(1, activation='sigmoid')  # 이진분류 (개, 고양이)                   
            ], name='classifier'
        )
    
    # 모델 호출 시 작동할 함수
    # 위에서 작성한 layer들이 어떤 순서로 불려야 하는지 모델 순서 정하기
    def call(self, input_tensor, training=False):
        x = self.conv_block_1(input_tensor)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x

In [ ]:
# None : batch axis 부분
# -> batch 값은 모델의 구조를 정하는데 중요하지 않아 일반적으로 None 적용
input_shape = (None, 256, 256, 3)

model = SimpleCNN()
model.build(input_shape)
model.summary()

### 4.4 Model compiling
- keras에서 학습에 사용할 optimizer, loss, 평가에 사용할 metric을 정해주는 과정

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics='accuracy'
)